In [ ]:
import os
import scipy.io
import numpy as np

def process_crowd_data(dataset_root_path):
    ground_truth_dir = os.path.join(dataset_root_path, 'ground_truth')
    formatted_dir = os.path.join(dataset_root_path, 'formatted')

    os.makedirs(formatted_dir, exist_ok=True)
    print(f"Ensured output directory exists: {formatted_dir}\n")

    if not os.path.exists(ground_truth_dir):
        print(f"Error: Ground truth directory not found: {ground_truth_dir}")
        print("Please ensure the path is correct and 'ground_truth' folder exists within your dataset root.")
        return

    mat_files = [f for f in os.listdir(ground_truth_dir) if f.endswith('.mat')]
    if not mat_files:
        print(f"No .mat files found in: {ground_truth_dir}")
        return

    print(f"Found {len(mat_files)} .mat files to process.")

    processed_count = 0
    failed_count = 0

    for filename in mat_files:
        mat_filepath = os.path.join(ground_truth_dir, filename)
        print(f"\nProcessing: {filename}")

        try:
            mat_contents = scipy.io.loadmat(mat_filepath)

            if 'image_info' in mat_contents:
                coordinates = mat_contents['image_info'][0,0][0,0][0]

                if isinstance(coordinates, np.ndarray) and coordinates.ndim == 2 and coordinates.shape[1] == 2:
                    print(f"   Successfully extracted coordinates.")
                    print(f"   Coordinates shape: {coordinates.shape}, Dtype: {coordinates.dtype}")

                    base_name = os.path.splitext(filename)[0]
                    output_name = base_name.replace('GT_', '')
                    output_filename = f"{output_name}.npy"
                    output_filepath = os.path.join(formatted_dir, output_filename)

                    np.save(output_filepath, coordinates)
                    print(f"   Saved coordinates to: {output_filepath}")
                    processed_count += 1
                else:
                    print(f"   Warning: Extracted 'image_info' content is not a (N, 2) NumPy array as expected.")
                    print(f"   Type: {type(coordinates)}, Shape: {getattr(coordinates, 'shape', 'N/A')}")
                    print(f"   This file might not contain coordinates in the expected format after indexing.")
                    failed_count += 1

            else:
                print(f"   Error: 'image_info' key not found in {filename}.")
                failed_count += 1

        except FileNotFoundError:
            print(f"   Error: File not found {mat_filepath}. Skipping.")
            failed_count += 1
        except KeyError as e:
            print(f"   Error: Missing key or invalid access during loading {filename}: {e}")
            print(f"   Please double-check the structure of your .mat file, especially the 'image_info' key.")
            failed_count += 1
        except IndexError as e:
            print(f"   Error: Indexing failed for {filename} with provided [0,0][0,0][0] path: {e}")
            print(f"   The nested structure might be different than expected for this specific file.")
            failed_count += 1
        except Exception as e:
            print(f"   An unexpected error occurred while processing {filename}: {e}")
            failed_count += 1

    print(f"\n--- Processing Summary ---")
    print(f"Total .mat files found: {len(mat_files)}")
    print(f"Successfully processed: {processed_count}")
    print(f"Failed to process: {failed_count}")
    print(f"Formatted coordinates saved to: {formatted_dir}")


dataset_base_path = "/content/crowd_wala_dataset/train_data"

process_crowd_data(dataset_base_path)

Ensured output directory exists: /content/crowd_wala_dataset/train_data/formatted

Found 400 .mat files to process.

Processing: GT_IMG_45.mat
  Successfully extracted coordinates.
  Coordinates shape: (30, 2), Dtype: float64
  Saved coordinates to: /content/crowd_wala_dataset/train_data/formatted/IMG_45.npy

Processing: GT_IMG_116.mat
  Successfully extracted coordinates.
  Coordinates shape: (168, 2), Dtype: float64
  Saved coordinates to: /content/crowd_wala_dataset/train_data/formatted/IMG_116.npy

Processing: GT_IMG_302.mat
  Successfully extracted coordinates.
  Coordinates shape: (82, 2), Dtype: float64
  Saved coordinates to: /content/crowd_wala_dataset/train_data/formatted/IMG_302.npy

Processing: GT_IMG_104.mat
  Successfully extracted coordinates.
  Coordinates shape: (31, 2), Dtype: float64
  Saved coordinates to: /content/crowd_wala_dataset/train_data/formatted/IMG_104.npy

Processing: GT_IMG_212.mat
  Successfully extracted coordinates.
  Coordinates shape: (111, 2), Dtyp

In [ ]:
import os
import numpy as np
from PIL import Image

def convert_npy_to_yolo_labels(dataset_root_path, fixed_box_width=20, fixed_box_height=40):
    images_dir = os.path.join(dataset_root_path, 'images')
    formatted_npy_dir = os.path.join(dataset_root_path, 'formatted')
    yolo_labels_dir = os.path.join(dataset_root_path, 'yolo_labels')

    os.makedirs(yolo_labels_dir, exist_ok=True)
    print(f"Ensured YOLO label directory exists: {yolo_labels_dir}\n")

    npy_files = [f for f in os.listdir(formatted_npy_dir) if f.endswith('.npy')]
    if not npy_files:
        print(f"No .npy files found in: {formatted_npy_dir}")
        return

    print(f"Found {len(npy_files)} .npy files to convert.")

    for npy_filename in npy_files:
        img_filename_base = os.path.splitext(npy_filename)[0]
        img_filepath = os.path.join(images_dir, f"{img_filename_base}.jpg")
        npy_filepath = os.path.join(formatted_npy_dir, npy_filename)
        label_filepath = os.path.join(yolo_labels_dir, f"{img_filename_base}.txt")

        if not os.path.exists(img_filepath):
            print(f"Warning: Corresponding image not found for {npy_filename} at {img_filepath}. Skipping.")
            continue

        try:
            with Image.open(img_filepath) as img:
                img_width, img_height = img.size

            coordinates = np.load(npy_filepath)

            if coordinates.ndim != 2 or coordinates.shape[1] != 2:
                print(f"Warning: {npy_filename} does not contain (N, 2) coordinates. Skipping.")
                continue

            yolo_lines = []
            for x_abs, y_abs in coordinates:
                x_center_abs = x_abs
                y_center_abs = y_abs

                x_center_norm = x_center_abs / img_width
                y_center_norm = y_center_abs / img_height
                width_norm = fixed_box_width / img_width
                height_norm = fixed_box_height / img_height

                x_center_norm = np.clip(x_center_norm, 0, 1)
                y_center_norm = np.clip(y_center_norm, 0, 1)
                width_norm = np.clip(width_norm, 0, 1)
                height_norm = np.clip(height_norm, 0, 1)

                yolo_lines.append(f"0 {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}")

            with open(label_filepath, 'w') as f:
                f.write("\n".join(yolo_lines))
            print(f"   Generated YOLO label: {label_filepath}")

        except Exception as e:
            print(f"   Error processing {npy_filename}: {e}. Skipping.")

    print("\n--- YOLO Label Conversion Complete ---")
    print(f"YOLO format label files are in: {yolo_labels_dir}")
    print("Next, you'll need to split your images and labels into train/val sets and create a data.yaml.")

dataset_path = "/content/crowd_wala_dataset/train_data"

BOX_WIDTH = 20
BOX_HEIGHT = 40

convert_npy_to_yolo_labels(dataset_path, fixed_box_width=BOX_WIDTH, fixed_box_height=BOX_HEIGHT)

Ensured YOLO label directory exists: /content/crowd_wala_dataset/train_data/yolo_labels

Found 400 .npy files to convert.
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_341.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_15.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_326.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_199.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_292.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_344.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_191.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_323.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_210.txt
  Generated YOLO label: /content/crowd_wala_dataset/train_data/yolo_labels/IMG_331.txt
  Generat

In [ ]:
import os
import shutil
import random

def split_yolo_dataset(dataset_root_path, val_split_ratio=0.2, seed=42):
    random.seed(seed)

    source_images_dir = os.path.join(dataset_root_path, 'images')
    source_labels_dir = os.path.join(dataset_root_path, 'yolo_labels')

    target_base_labels_dir = os.path.join(dataset_root_path, 'labels')
    target_images_train_dir = os.path.join(source_images_dir, 'train')
    target_images_val_dir = os.path.join(source_images_dir, 'val')
    target_labels_train_dir = os.path.join(target_base_labels_dir, 'train')
    target_labels_val_dir = os.path.join(target_base_labels_dir, 'val')

    os.makedirs(target_images_train_dir, exist_ok=True)
    os.makedirs(target_images_val_dir, exist_ok=True)
    os.makedirs(target_labels_train_dir, exist_ok=True)
    os.makedirs(target_labels_val_dir, exist_ok=True)

    print(f"Dataset root: {dataset_root_path}")
    print(f"Source images: {source_images_dir}")
    print(f"Source labels: {source_labels_dir}")
    print(f"Target images train: {target_images_train_dir}")
    print(f"Target images val: {target_images_val_dir}")
    print(f"Target labels train: {target_labels_train_dir}")
    print(f"Target labels val: {target_labels_val_dir}\n")

    all_image_files = [f for f in os.listdir(source_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')) and not os.path.isdir(os.path.join(source_images_dir, f))]

    all_image_files = [f for f in all_image_files if f not in ['train', 'val']]

    if not all_image_files:
        print(f"No image files found in {source_images_dir}. Please ensure images are directly in this folder.")
        return

    print(f"Found {len(all_image_files)} image files to split.")

    random.shuffle(all_image_files)

    num_val = int(len(all_image_files) * val_split_ratio)

    if len(all_image_files) < 2:
        print("Warning: Not enough images to split. All images will be placed in the training set.")
        train_files = all_image_files
        val_files = []
    elif num_val == 0:
        if len(all_image_files) > 0:
            print("Warning: Validation split ratio is too small for the number of images. Setting at least one validation image.")
            num_val = 1
        train_files = all_image_files[num_val:]
        val_files = all_image_files[:num_val]
    elif num_val == len(all_image_files):
        print("Warning: Validation split ratio is too high. Setting at least one training image.")
        num_val = len(all_image_files) - 1
        train_files = all_image_files[num_val:]
        val_files = all_image_files[:num_val]
    else:
        train_files = all_image_files[num_val:]
        val_files = all_image_files[:num_val]


    print(f"Splitting into {len(train_files)} training images and {len(val_files)} validation images.")

    def move_files(files_list, target_img_dir, target_lbl_dir):
        for img_filename in files_list:
            base_name = os.path.splitext(img_filename)[0]
            label_filename = f"{base_name}.txt"

            source_img_path = os.path.join(source_images_dir, img_filename)
            source_label_path = os.path.join(source_labels_dir, label_filename)

            target_img_path = os.path.join(target_img_dir, img_filename)
            target_label_path = os.path.join(target_lbl_dir, label_filename)

            try:
                shutil.move(source_img_path, target_img_path)
                if os.path.exists(source_label_path):
                    shutil.move(source_label_path, target_label_path)
                else:
                    print(f"   Warning: Label file {label_filename} not found for image {img_filename}. Skipping label move.")
            except Exception as e:
                print(f"   Error moving {img_filename} or its label: {e}")

    print("\nMoving training files...")
    move_files(train_files, target_images_train_dir, target_labels_train_dir)
    print("Moving validation files...")
    move_files(val_files, target_images_val_dir, target_labels_val_dir)

    print("\n--- Data Splitting Complete ---")
    print("Your dataset is now organized for YOLOv8 training:")
    print(f"   Images: {target_images_train_dir} and {target_images_val_dir}")
    print(f"   Labels: {target_labels_train_dir} and {target_labels_val_dir}")
    print("\nRemember to create your data.yaml file pointing to these new 'train' and 'val' directories for images and labels.")
    print("Example data.yaml content:")
    print(f"""
train: {os.path.relpath(target_images_train_dir, start=os.path.dirname(os.path.dirname(target_images_train_dir)))}
val: {os.path.relpath(target_images_val_dir, start=os.path.dirname(os.path.dirname(target_images_val_dir)))}

nc: 1
names: ['person']
""")

dataset_path_to_split = "/content/crowd_wala_dataset/train_data"

split_yolo_dataset(dataset_path_to_split, val_split_ratio=0.2, seed=42)

Dataset root: /content/crowd_wala_dataset/train_data
Source images: /content/crowd_wala_dataset/train_data/images
Source labels: /content/crowd_wala_dataset/train_data/yolo_labels
Target images train: /content/crowd_wala_dataset/train_data/images/train
Target images val: /content/crowd_wala_dataset/train_data/images/val
Target labels train: /content/crowd_wala_dataset/train_data/labels/train
Target labels val: /content/crowd_wala_dataset/train_data/labels/val

Found 400 image files to split.
Splitting into 320 training images and 80 validation images.

Moving training files...
Moving validation files...

--- Data Splitting Complete ---
Your dataset is now organized for YOLOv8 training:
  Images: /content/crowd_wala_dataset/train_data/images/train and /content/crowd_wala_dataset/train_data/images/val
  Labels: /content/crowd_wala_dataset/train_data/labels/train and /content/crowd_wala_dataset/train_data/labels/val

Remember to create your data.yaml file pointing to these new 'train' and

In [ ]:
import os

def create_yolo_data_yaml(dataset_main_folder, num_classes, class_names, yaml_filename="data.yaml"):
    yaml_filepath = os.path.join(dataset_main_folder, yaml_filename)

    train_images_path_relative = os.path.join('train_data', 'images', 'train')
    val_images_path_relative = os.path.join('train_data', 'images', 'val')

    yaml_content = f"""
path: {dataset_main_folder}

train: {train_images_path_relative}
val: {val_images_path_relative}

nc: {num_classes}

names: {class_names}
"""
    yaml_content = yaml_content.strip() + "\n"

    try:
        with open(yaml_filepath, 'w') as f:
            f.write(yaml_content)
        print(f"Successfully created '{yaml_filename}' at: {yaml_filepath}")
        print("\nContent of data.yaml:")
        print(yaml_content)
    except Exception as e:
        print(f"Error creating {yaml_filename}: {e}")

dataset_main_folder_path = "/content/crowd_wala_dataset/"

num_classes = 1
class_names = ['person']

create_yolo_data_yaml(dataset_main_folder_path, num_classes, class_names)

Successfully created 'data.yaml' at: /content/crowd_wala_dataset/data.yaml

Content of data.yaml:
# YOLOv8 Dataset Configuration
path: /content/crowd_wala_dataset/ # Dataset root directory (optional, can be relative to current working directory during training)

train: train_data/images/train
val: train_data/images/val

# Number of classes
nc: 1

# Class names
names: ['person']



In [ ]:
from ultralytics import YOLO
import os

def finetune_yolov8_model(
    model_name='yolov8n.pt',
    data_yaml_path="/content/crowd_wala_dataset/data.yaml",
    epochs=20,
    imgsz=640,
    batch_size=-1,
    device='cpu',
    optimizer='AdamW',
    lr0=0.001,
    freeze=10
):
    print(f"--- Starting YOLOv8 Finetuning ---")
    print(f"Model: {model_name}")
    print(f"Data YAML: {data_yaml_path}")
    print(f"Epochs: {epochs}")
    print(f"Image Size: {imgsz}")
    print(f"Batch Size: {batch_size}")
    print(f"Device: {device}")


    try:
        model = YOLO(model_name)
        print(f"\nSuccessfully loaded pretrained model: {model_name}")
    except Exception as e:
        print(f"Error loading model {model_name}: {e}")
        print("Please ensure the model name is correct and you have an internet connection to download it, or it's available locally.")
        return

    try:
        if not os.path.exists(data_yaml_path):
            print(f"Error: data.yaml not found at {data_yaml_path}.")
            print("Please ensure the path to your data.yaml file is correct.")
            return

        results = model.train(
            data=data_yaml_path,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch_size,
            device=device,
            optimizer=optimizer,
            lr0=lr0,
            freeze=freeze
        )
        print("\n--- YOLOv8 Finetuning Completed ---")
        print("Training results can be found in the 'runs/detect' directory (or your specified project/name directory).")
        print(f"The best model weights are typically at: {model.trainer.save_dir}/weights/best.pt")

    except Exception as e:
        print(f"An error occurred during training: {e}")
        print("Common issues: Incorrect data.yaml path, GPU memory issues, incompatible data format.")


my_data_yaml_path = "/content/crowd_wala_dataset/data.yaml"

finetune_yolov8_model(
    model_name='yolov8m.pt',
    data_yaml_path=my_data_yaml_path,
    epochs=50,
    imgsz=640,
    batch_size=16,
    device=0,
    optimizer='AdamW',
    lr0=0.001,
    freeze=10
)

--- Starting YOLOv8 Finetuning ---
Model: yolo11n.pt
Data YAML: /content/crowd_wala_dataset/data.yaml
Epochs: 50
Image Size: 640
Batch Size: 16
Device: 0

Successfully loaded pretrained model: yolo11n.pt
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/crowd_wala_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n

train: Scanning /content/crowd_wala_dataset/train_data/labels/train.cache... 320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 320/320 [00:00<?, ?it/s]

train: /content/crowd_wala_dataset/train_data/images/train/IMG_133.jpg: 2 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_134.jpg: 1 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_212.jpg: 1 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_263.jpg: 2 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_272.jpg: 2 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_303.jpg: 1 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_349.jpg: 3 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_353.jpg: 1 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_361.jpg: 1 duplicate labels removed
train: /content/crowd_wala_dataset/train_data/images/train/IMG_370.jpg: 4 duplicate labels removed
train: /co

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 982.2±689.2 MB/s, size: 152.4 KB)


val: Scanning /content/crowd_wala_dataset/train_data/labels/val.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]

...

      50/50      6.58G       1.72      1.045     0.9473       1439        640: 100%|██████████| 20/20 [00:07<00:00,  2.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.19it/s]

                   all         80       9948      0.718      0.623      0.685       0.35



50 epochs completed in 0.127 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 5.5MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:06<00:00,  2.14s/it]


                   all         80       9948      0.718      0.625      0.685       0.35
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/detect/train7

--- YOLOv8 Finetuning Completed ---
Training results can be found in the 'runs/detect' directory (or your specified project/name directory).
The best model weights are typically at: runs/detect/train7/weights/best.pt


In [ ]:
import os
import scipy.io
import numpy as np
from PIL import Image
import shutil
import random

def process_mat_files_to_npy(base_data_path):
    ground_truth_dir = os.path.join(base_data_path, 'ground_truth')
    formatted_dir = os.path.join(base_data_path, 'formatted')

    os.makedirs(formatted_dir, exist_ok=True)
    print(f"Ensured output directory exists: {formatted_dir}\n")

    if not os.path.exists(ground_truth_dir):
        print(f"Error: Ground truth directory not found: {ground_truth_dir}")
        print("Please ensure the path is correct and 'ground_truth' folder exists within your base data path.")
        return

    mat_files = [f for f in os.listdir(ground_truth_dir) if f.endswith('.mat')]
    if not mat_files:
        print(f"No .mat files found in: {ground_truth_dir}")
        return

    print(f"Found {len(mat_files)} .mat files to process in {base_data_path}.")

    processed_count = 0
    failed_count = 0

    for filename in mat_files:
        mat_filepath = os.path.join(ground_truth_dir, filename)

        try:
            mat_contents = scipy.io.loadmat(mat_filepath)

            if 'image_info' in mat_contents:
                coordinates = mat_contents['image_info'][0,0][0,0][0]

                if isinstance(coordinates, np.ndarray) and coordinates.ndim == 2 and coordinates.shape[1] == 2:
                    base_name = os.path.splitext(filename)[0]
                    output_name = base_name.replace('GT_', '')
                    output_filename = f"{output_name}.npy"
                    output_filepath = os.path.join(formatted_dir, output_filename)

                    np.save(output_filepath, coordinates)
                    processed_count += 1
                else:
                    print(f"   Warning: Extracted 'image_info' content from {filename} is not a (N, 2) NumPy array. Skipping.")
                    failed_count += 1
            else:
                print(f"   Error: 'image_info' key not found in {filename}. Skipping.")
                failed_count += 1

        except FileNotFoundError:
            print(f"   Error: File not found {mat_filepath}. Skipping.")
            failed_count += 1
        except KeyError as e:
            print(f"   Error: Missing key or invalid access during loading {filename}: {e}. Skipping.")
            failed_count += 1
        except IndexError as e:
            print(f"   Error: Indexing failed for {filename} with provided [0,0][0,0][0] path: {e}. Skipping.")
            failed_count += 1
        except Exception as e:
            print(f"   An unexpected error occurred while processing {filename}: {e}. Skipping.")
            failed_count += 1

    print(f"\n--- Summary for .mat to .npy in {base_data_path} ---")
    print(f"Total .mat files found: {len(mat_files)}")
    print(f"Successfully processed: {processed_count}")
    print(f"Failed to process: {failed_count}")
    print(f"Formatted coordinates saved to: {formatted_dir}")


def convert_npy_to_yolo_labels(base_data_path, fixed_box_width=20, fixed_box_height=40):
    original_images_dir = os.path.join(base_data_path, 'images')
    formatted_npy_dir = os.path.join(base_data_path, 'formatted')
    final_labels_dir = os.path.join(base_data_path, 'labels')

    os.makedirs(final_labels_dir, exist_ok=True)

    print(f"\n--- Starting .npy to YOLO Labels for {base_data_path} ---")
    print(f"Reading NPY from: {formatted_npy_dir}")
    print(f"Saving Labels to: {final_labels_dir}")
    print(f"Using fixed box size: {fixed_box_width}x{fixed_box_height} pixels\n")

    npy_files = [f for f in os.listdir(formatted_npy_dir) if f.endswith('.npy')]
    if not npy_files:
        print(f"No .npy files found in: {formatted_npy_dir}. Skipping label conversion.")
        return

    processed_count = 0
    failed_count = 0

    for npy_filename in npy_files:
        img_filename_base = os.path.splitext(npy_filename)[0]
        img_filename_jpg = f"{img_filename_base}.jpg"

        current_img_filepath = os.path.join(original_images_dir, img_filename_jpg)
        npy_filepath = os.path.join(formatted_npy_dir, npy_filename)
        label_filepath = os.path.join(final_labels_dir, f"{img_filename_base}.txt")

        if not os.path.exists(current_img_filepath):
            print(f"Warning: Corresponding image not found for {npy_filename} at {current_img_filepath}. Skipping.")
            failed_count += 1
            continue

        try:
            with Image.open(current_img_filepath) as img:
                img_width, img_height = img.size

            coordinates = np.load(npy_filepath)

            if coordinates.ndim != 2 or coordinates.shape[1] != 2:
                print(f"Warning: {npy_filename} does not contain (N, 2) coordinates. Skipping.")
                failed_count += 1
                continue

            yolo_lines = []
            for x_abs, y_abs in coordinates:
                x_center_abs = x_abs
                y_center_abs = y_abs

                x_center_norm = x_center_abs / img_width
                y_center_norm = y_abs / img_height
                width_norm = fixed_box_width / img_width
                height_norm = fixed_box_height / img_height

                x_center_norm = np.clip(x_center_norm, 0, 1)
                y_center_norm = np.clip(y_center_norm, 0, 1)
                width_norm = np.clip(width_norm, 0, 1)
                height_norm = np.clip(height_norm, 0, 1)

                yolo_lines.append(f"0 {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}")

            with open(label_filepath, 'w') as f:
                f.write("\n".join(yolo_lines))
            processed_count += 1

        except Exception as e:
            print(f"   Error processing {npy_filename}: {e}. Skipping.")
            failed_count += 1

    print(f"\n--- Summary for .npy to YOLO Labels in {base_data_path} ---")
    print(f"Successfully converted: {processed_count}")
    print(f"Failed conversions: {failed_count}")
    print(f"YOLO format label files are in: {final_labels_dir}")


def create_yolo_data_yaml(dataset_main_folder, num_classes, class_names, yaml_filename="data.yaml"):
    yaml_filepath = os.path.join(dataset_main_folder, yaml_filename)

    train_images_path_relative = os.path.join('train_data', 'images', 'train')
    val_images_path_relative = os.path.join('train_data', 'images', 'val')
    test_images_path_relative = os.path.join('test_data', 'images')

    yaml_content = f"""
path: {dataset_main_folder}

train: {train_images_path_relative}
val: {val_images_path_relative}
test: {test_images_path_relative}

nc: {num_classes}

names: {class_names}
"""
    yaml_content = yaml_content.strip() + "\n"

    try:
        with open(yaml_filepath, 'w') as f:
            f.write(yaml_content)
        print(f"\nSuccessfully created '{yaml_filename}' at: {yaml_filepath}")
        print("\nContent of data.yaml:")
        print(yaml_content)
    except Exception as e:
        print(f"Error creating {yaml_filename}: {e}")

DATASET_MAIN_ROOT = "/content/crowd_wala_dataset/"

TEST_DATA_FOLDER = os.path.join(DATASET_MAIN_ROOT, "test_data")

BOX_WIDTH = 20
BOX_HEIGHT = 40

NUM_CLASSES = 1
CLASS_NAMES = ['person']

print("\n========== Processing TEST DATA ==========")
print(f"Working on: {TEST_DATA_FOLDER}")
process_mat_files_to_npy(TEST_DATA_FOLDER)
convert_npy_to_yolo_labels(TEST_DATA_FOLDER, fixed_box_width=BOX_WIDTH, fixed_box_height=BOX_HEIGHT)

print("\n========== Creating data.yaml ==========")
create_yolo_data_yaml(DATASET_MAIN_ROOT, NUM_CLASSES, CLASS_NAMES)

print("\n--- Test Data Preprocessing and data.yaml Creation Completed ---")
print("Your test dataset is now prepared for YOLOv8 testing.")
print("The data.yaml file has been created/updated with the test data path.")


========== Processing TEST DATA ==========
Working on: /content/crowd_wala_dataset/test_data
Ensured output directory exists: /content/crowd_wala_dataset/test_data/formatted

Found 316 .mat files to process in /content/crowd_wala_dataset/test_data.

--- Summary for .mat to .npy in /content/crowd_wala_dataset/test_data ---
Total .mat files found: 316
Successfully processed: 316
Failed to process: 0
Formatted coordinates saved to: /content/crowd_wala_dataset/test_data/formatted

--- Starting .npy to YOLO Labels for /content/crowd_wala_dataset/test_data ---
Reading NPY from: /content/crowd_wala_dataset/test_data/formatted
Saving Labels to: /content/crowd_wala_dataset/test_data/labels
Using fixed box size: 20x40 pixels


--- Summary for .npy to YOLO Labels in /content/crowd_wala_dataset/test_data ---
Successfully converted: 316
Failed conversions: 0
YOLO format label files are in: /content/crowd_wala_dataset/test_data/labels

========== Creating data.yaml ==========

Successfully created 

In [ ]:
import os
from ultralytics import YOLO

DATASET_MAIN_ROOT = "/content/crowd_wala_dataset/"
DATA_YAML_PATH = os.path.join(DATASET_MAIN_ROOT, "data.yaml")
FINETUNED_MODEL_PATH = "runs/detect/train7/weights/best.pt"
IMGSZ = 640
DEVICE = 0

In [ ]:
import os
from ultralytics import YOLO

print("\n========== Starting Model Evaluation on Test Set ==========")
print(f"Loading model from: {FINETUNED_MODEL_PATH}")
print(f"Using data configuration from: {DATA_YAML_PATH}")

try:
    model = YOLO(FINETUNED_MODEL_PATH)

    metrics = model.val(
        data=DATA_YAML_PATH,
        imgsz=IMGSZ,
        batch=16,
        device=DEVICE,
        split='test'
    )

    print("\n--- Evaluation Results ---")
    print(f"Metrics object type: {type(metrics)}")
    print(f"mAP50-95: {metrics.box.map}")
    print(f"mAP50: {metrics.box.map50}")
    print(f"mAP75: {metrics.box.map75}")
    print(f"Precision: {metrics.box.mp}")
    print(f"Recall: {metrics.box.mr}")

except FileNotFoundError:
    print(f"Error: Model weights not found at {FINETUNED_MODEL_PATH}. Please check the path.")
except Exception as e:
    print(f"An error occurred during model evaluation: {e}")

print("\n--- Model Evaluation Completed ---")


========== Starting Model Evaluation on Test Set ==========
Loading model from: runs/detect/train7/weights/best.pt
Using data configuration from: /content/crowd_wala_dataset/data.yaml
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2375.8±824.9 MB/s, size: 140.8 KB)


val: Scanning /content/crowd_wala_dataset/test_data/labels.cache... 316 images, 0 backgrounds, 0 corrupt: 100%|██████████| 316/316 [00:00<?, ?it/s]

val: /content/crowd_wala_dataset/test_data/images/IMG_109.jpg: 12 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_116.jpg: 1 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_123.jpg: 1 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_131.jpg: 2 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_138.jpg: 1 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_154.jpg: 1 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_212.jpg: 1 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_219.jpg: 2 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_253.jpg: 1 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_314.jpg: 1 duplicate labels removed
val: /content/crowd_wala_dataset/test_data/images/IMG_33.jpg: 2 duplicate labels removed
val: /cont


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:19<00:00,  1.02it/s]


                   all        316      39178      0.737      0.652      0.716       0.37
Speed: 1.5ms preprocess, 6.0ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to runs/detect/val8

--- Evaluation Results ---
Metrics object type: <class 'ultralytics.utils.metrics.DetMetrics'>
mAP50-95: 0.36971871207082374
mAP50: 0.7159097905349217
mAP75: 0.3312741170679686
Precision: 0.7371874470336324
Recall: 0.6522027668589514

--- Model Evaluation Completed ---
